In [8]:
%load_ext autoreload
%autoreload 2

In [4]:
!pip install evaluate

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/mvkairov/bhwdl.git

Cloning into 'bhwdl'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 102 (delta 48), reused 80 (delta 26), pack-reused 0
Receiving objects: 100% (102/102), 397.26 KiB | 9.24 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [3]:
%cd bhwdl

/kaggle/working/bhwdl


In [10]:
import sys
sys.path.append('LLAMA')

import torch
import evaluate

In [35]:
from LLAMA.tinystories import TSDataset
from LLAMA.model import LM, make_seq

In [13]:
eval_ds = TSDataset('bpe.vocab')

In [18]:
%ls ../../

input/  lib/  working/


In [22]:
model = LM(8, 512, 8, eval_ds.vocab_size, 1536, 0.08)
model.load_state_dict(torch.load('../../input/lm-model/best_model/best_model.pt', 'cuda'))
model = model.to('cuda')

36395672 parameters :)


In [33]:
torch.load('../../input/lm-model/best_model/best_model.pt', 'cuda').keys()

odict_keys(['embedding.weight', 'decoder.decoder.0.attention.in_proj_weight', 'decoder.decoder.0.attention.in_proj_bias', 'decoder.decoder.0.attention.out_proj.weight', 'decoder.decoder.0.attention.out_proj.bias', 'decoder.decoder.0.ff.0.weight', 'decoder.decoder.0.ff.0.bias', 'decoder.decoder.0.ff.3.weight', 'decoder.decoder.0.ff.3.bias', 'decoder.decoder.0.att_lnorm.weight', 'decoder.decoder.0.att_lnorm.bias', 'decoder.decoder.0.ff_lnorm.weight', 'decoder.decoder.0.ff_lnorm.bias', 'decoder.decoder.1.attention.in_proj_weight', 'decoder.decoder.1.attention.in_proj_bias', 'decoder.decoder.1.attention.out_proj.weight', 'decoder.decoder.1.attention.out_proj.bias', 'decoder.decoder.1.ff.0.weight', 'decoder.decoder.1.ff.0.bias', 'decoder.decoder.1.ff.3.weight', 'decoder.decoder.1.ff.3.bias', 'decoder.decoder.1.att_lnorm.weight', 'decoder.decoder.1.att_lnorm.bias', 'decoder.decoder.1.ff_lnorm.weight', 'decoder.decoder.1.ff_lnorm.bias', 'decoder.decoder.2.attention.in_proj_weight', 'decoder.d

In [41]:
raw_gen = make_seq(model, eval_ds.sp_model, eval_ds.pad_id, 32)
eval_txt = eval_ds.ids2text(raw_gen)

/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(


In [42]:
eval_txt

['once upon a time there was a little girl called lily. she was very brave and adventurous. one day she was out in the garden when she stumbled across a big, white flower. she smiled and said to herself, "this flower looks so good!"suddenly she saw a figure appeared in the garden. it was a tall man. lily said, "i want to know how you can grow up to know how to bloom?" the man said, "not right now, lily. it\'s not possible." so lily held the book and the man told her, "this was my paper. you can make it very special. it\'s a very special letter". lily felt really proud of her letter. she did it all by herself and enjoyed taking it to the fair. the moral of the story is to be happy with what you have and take your special care of. with your heart can be good too, like the kind elders in the story. and kindness can be a wonderful thing  ⁇  skill can accomplish when you te it too. with the voice in the story may disappear, settling just believing in the sweet voice. and even though the man

In [52]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [53]:
perplexity = evaluate.load('perplexity', module_type='metric')
p = perplexity.compute(model_id='gpt2-xl',
                             add_start_token=False,
                             predictions=eval_txt)

  0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 164.00 MiB (GPU 0; 15.90 GiB total capacity; 12.82 GiB already allocated; 181.75 MiB free; 14.90 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [51]:
p

{'perplexities': [56.38043212890625,
  36.248077392578125,
  33.661170959472656,
  83.74586486816406,
  26.87843132019043,
  39.96218490600586,
  37.61302185058594,
  48.04453659057617,
  42.1041145324707,
  45.7709846496582,
  46.41226577758789,
  36.39927673339844,
  51.76651382446289,
  28.759172439575195,
  24.84056854248047,
  32.92829513549805,
  47.51721954345703,
  85.750244140625,
  31.293060302734375,
  42.9622802734375,
  25.71197509765625,
  41.6978645324707,
  33.47461700439453,
  35.8414192199707,
  38.241912841796875,
  35.9149169921875,
  61.73870086669922,
  34.19203186035156,
  31.010656356811523,
  34.558502197265625,
  56.4384765625,
  53.957969665527344],
 'mean_perplexity': 42.55677372217178}

да и хрен с ним